In [2]:
# import the necessary packages
from imutils import face_utils
from datetime import datetime
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

class Eye:
    left = [0, 0]
    right = [0, 0]
    top = [0, 0]
    bottom = [0, 0]

    corner_left_bot = [0, 0]
    corner_right_top = [0, 0]

    center = [0, 0]

    side = ''

    def __init__(self, p1, p2, side):
        self.left = p2
        self.right = p1

        med_x = int((p1[0] + p2[0]) / 2)
        med_y = int((p1[1] + p2[1]) / 2)

        self.center = [med_x, med_y]

        diff_dist = self.right[0] - med_x

        self.top = [med_x, med_y + diff_dist]
        self.bottom = [med_x, med_y - diff_dist]

        self.side = side

        self.corner_left_bot = [self.left[0] - 10, med_y + diff_dist]
        self.corner_right_top = [self.right[0] + 10, med_y - diff_dist]

    def draw(self, frame):
        return cv2.rectangle(frame, (self.corner_left_bot[0], self.corner_left_bot[1]), (self.corner_right_top[0], self.corner_right_top[1]),
                             (255, 255, 255), cv2.FILLED)

    def identify(self, frame):
        cv2.putText(frame, self.side, (self.center[0] - 10, self.center[1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
        cv2.circle(frame, tuple(self.left), 1, (0, 0, 255), -1)
        cv2.circle(frame, tuple(self.right), 1, (0, 0, 255), -1)
        cv2.rectangle(frame, (self.corner_left_bot[0], self.corner_left_bot[1]), (self.corner_right_top[0], self.corner_right_top[1]),
                      (0, 255, 0), 1)

        cv2.circle(frame, tuple(self.corner_right_top), 1, (0, 0, 255), -1)
        cv2.circle(frame, tuple(self.corner_left_bot), 1, (255, 0, 0), -1)

In [7]:
def non_linear_rootMod2(img, a, b):
    
    img_copy = img.copy().astype(np.float32)/255.0
    res_a = cv2.pow(img_copy,0.5)
    res_a = cv2.multiply(res_a, a)
    res = cv2.add(res_a,b)
    
    res[res<0] = 0
    res = res*255.0
    res[res>255] = 255
    
    res = res.astype(np.uint8)
    res = cv2.medianBlur(res,5)
    
    return res

def total(img, threshold_low_lim, multiply): 
    a = 1.3
    b = -0.1
    params = [a,b]
    res = non_linear_rootMod2(img, a, b)
    img_mult = np.zeros(res.shape, np.uint8)
    
    multiply = multiply / 10
    
    img_mult = cv2.multiply(res,multiply)
    
    rows, cols = img.shape
    
    ret, res = cv2.threshold(img_mult,threshold_low_lim,255,cv2.THRESH_BINARY_INV)
    
    img, contours, _ = cv2.findContours(res, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contours =sorted(contours, key=lambda x: cv2.contourArea(x), reverse = True )
    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)
        #cv2.drawContours(img, [cnt], -1, (0, 0, 255), 1)
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 1 )
        cv2.line(img, (x + int(w/2), 0), (x + int(w/2),rows), (0, 255, 0), 1)
        cv2.line(img,( 0, y + int(h/2)), (cols,y + int(h/2)),(0,255,0),1)
        break
    
    return res


def nothing(*arg):
    pass

def captureAndApplyTotal(f, fparams=[], path=''):

    print("[INFO] loading facial landmark predictor...")
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

    if len(path) > 0:
        # initialize the video stream from a video file
        print("[INFO] loading video file...")
        vs = cv2.VideoCapture(path)
        time.sleep(1.0)
    else:
        # initialize the video stream and sleep for a bit, allowing the
        # camera sensor to warm up
        print("[INFO] camera sensor warming up...")
        vs = cv2.VideoCapture(0)
        time.sleep(2.0)
        
    cv2.namedWindow("Trackbars")
    cv2.createTrackbar("threshold", "Trackbars", 0, 255, nothing)
    cv2.createTrackbar("multiply", "Trackbars", 5, 20, nothing)

    while True:
        
        # grab the frame from the threaded video stream, resize it to
        # have a maximum width of 400 pixels, and convert it to
        # grayscale
        ret, frame = vs.read()
        
        if not ret:
            break
        
        frame = imutils.resize(frame, width=400)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        frame = cv2.flip(frame, 1)
        gray = cv2.flip(gray, 1)
        # detect faces in the grayscale frame
        rects = detector(gray, 0)

        # loop over the face detections
        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)[0:4]

            eye_right = Eye(shape[0], shape[1], 'right')
            eye_left = Eye(shape[3], shape[2], 'left')

            eye_left.identify(frame)
            eye_right.identify(frame)

            oneEye = gray[eye_right.corner_right_top[1]:eye_right.corner_left_bot[1],
                     eye_right.corner_left_bot[0]:eye_right.corner_right_top[0]]
            
            
            threshold_low_lim = cv2.getTrackbarPos("threshold", "Trackbars")
            multiply = cv2.getTrackbarPos("multiply", "Trackbars")
            
            fparams = [threshold_low_lim, multiply]
            
            # Apply fn
            oneEyeTh = f(oneEye, *fparams)
            oneEyeTh = cv2.resize(oneEyeTh, None, fx=5, fy=5, interpolation=cv2.INTER_LINEAR)

            
            oneEye = cv2.resize(oneEye, None, fx=5, fy=5, interpolation=cv2.INTER_LINEAR)
            cv2.imshow("Cropped", oneEye)
            cv2.imshow("Cropped Filter", oneEyeTh)

        # show the frame
        cv2.imshow("frame", frame)

        key = cv2.waitKey(30) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    # do a bit of cleanup
    cv2.destroyAllWindows()
    vs.release()

In [8]:
# path = "" => captura la webcam | path = "video/file/path/video.mp4" => captura de un archivo de video
# Demo files: demo.mp4 - muestra.mp4
#path = "demo5.mp4"
#path = "demo.mp4"
#path = "muestra.mp4"
#path = "blue_eyes1.mp4"
path = ""

captureAndApplyTotal(total, path=path)

[INFO] loading facial landmark predictor...
[INFO] camera sensor warming up...
0
0
0
0
0
0
0
0
0
0
0
0
5
10
15
25
30
30
30
40
40
35
30
30
30
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
